In [1]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
df = pd.read_csv('data/cannabis.csv')

In [3]:
# df = df.replace('None', np.nan)
# df = df.dropna()

In [4]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
dtm = tfidf.fit_transform(df['Effects'])
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)

In [5]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [6]:
# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
NN_model = pickle.load(model_pkl)
print ("Loaded model :: ", NN_model)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [7]:
import pickle
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [8]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [9]:
import json
def recommend(user_input):
    temp_df = NN_model.kneighbors(tfidf_model.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.loc[temp_df[0][i]]['Strain']
        info_effects = df.loc[temp_df[0][i]]['Effects']
        info_flavor = df.loc[temp_df[0][i]]['Flavor']
        info_description = df.loc[temp_df[0][i]]['Description']
        info_rating = df.loc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [11]:
recommend('Hungry,Aroused,Sleepy,Giggly,Happy')

"Taliban-Poison"
"Hungry,Aroused,Sleepy,Giggly,Happy"
"Diesel,Skunk,Pine"
NaN
3.7
"Alien-Hallucination"
"Happy,Relaxed,Giggly,Aroused,Hungry"
"Sweet,Earthy,Diesel"
"Alien Hallucination is a hybrid cannabis strain that is believed to be a cross between the sedating Tahoe Alien and LSD, a classic Amsterdam hybrid known for its trippy head buzz. Bred by Alien Genetics, this strain will abduct the mind while your body remains anchored in deep physical relaxation."
4.1
"Strawberry-Frost"
"Happy,Aroused,Relaxed,Giggly,Hungry"
"Sweet,Strawberry,Earthy"
"Strawberry Frost by MaddFarmer is a three-way cross of fruity trichome producers. The genetic descendant of a Strawberry Cough/Deep Chunk hybrid crossed with Space Queen, Strawberry Frost is a heady indica-dominant strain that is gentle on the body. This dreamy strain calms the body and elevates the mind, placing the consumer in a relaxed, whimsical state. These effects naturally aid with stress, anxiety, and general restlessness.\u00a0"
5.0
"